<a href="https://colab.research.google.com/github/degar405/ExperimentacoesPreDiagnosticoGlaucoma/blob/main/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 27.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3


In [2]:
from google.colab import drive
from os import listdir
from skimage import io
import pywt
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import graycomatrix

drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

def exibirImagensCoeficientesWavelet(coeffs):
  titles = ['Approximation', ' Horizontal detail',
          'Vertical detail', 'Diagonal detail']
  LL, (LH, HL, HH) = coeffs

  fig = plt.figure(figsize=(12, 3))
  for i, a in enumerate([LL, LH, HL, HH]):
      ax = fig.add_subplot(1, 4, i + 1)
      ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
      ax.set_title(titles[i], fontsize=10)
      ax.set_xticks([])
      ax.set_yticks([])

  fig.tight_layout()
  plt.show()

def separarCanaisRGB(imagem):
  canalR = []
  canalG = []
  canalB = []

  for linha in imagem:
    linhaR = []
    linhaG = []
    linhaB = []

    for pixel in linha:
      linhaR.append(pixel[0])
      linhaG.append(pixel[1])
      linhaB.append(pixel[2])

    canalR.append(linhaR)
    canalG.append(linhaG)
    canalB.append(linhaB)
  
  return (np.array(canalR), np.array(canalG), np.array(canalB))

def aplicarWavelet(imagem, implementacao):
  return pywt.dwt2(imagem, implementacao)

def combinarImagensEmTonsDeCinza(R, G, B):
  imagem = []
  quantidadeLinhas = len(R)
  quantidadeColunas = len(R[0])

  for nLinha in range(0, quantidadeLinhas):
    linha = []

    for nColuna in range(0, quantidadeColunas):
      pixel = round(0.2126 * R[nLinha][nColuna] + 0.7154 * G[nLinha][nColuna] + 0.0721 * B[nLinha][nColuna])
      linha.append(pixel)

    imagem.append(linha)

  return imagem

def combinarCoeficientesWavelet(coeficientesR, coeficientesG, coeficientesB):
  (LLR, (LHR, HLR, HHR)) = coeficientesR
  (LLG, (LHG, HLG, HHG)) = coeficientesG
  (LLB, (LHB, HLB, HHB)) = coeficientesB

  LL = combinarImagensEmTonsDeCinza(LLR, LLG, LLB)
  LH = combinarImagensEmTonsDeCinza(LHR, LHG, LHB)
  HL = combinarImagensEmTonsDeCinza(HLR, HLG, HLB)
  HH = combinarImagensEmTonsDeCinza(HHR, HHG, HHB)

  return (LL, (LH, HL, HH))

def aplicarGlcm(imagem):
  distancias = [1]
  direcoes = [0, np.pi/4, np.pi/2, 3*np.pi/4]
  resultadoGlcm = graycomatrix(np.array(imagem, dtype=np.uint8), distancias, direcoes)

  qtdDirecoes = len(direcoes)
  matrizes = []
  for i in range(0, qtdDirecoes):
    matrizes.append(resultadoGlcm[:, :, 0, i])

  niveis = len(matrizes[0])
  matrizRetorno = []
  for nLinha in range(0, niveis):
    linha = []
    for nColuna in range(0, niveis):
      acumuladorFrequencia = 0
      for matriz in matrizes:
        acumuladorFrequencia += matriz[nLinha, nColuna]
      linha.append(round(acumuladorFrequencia/qtdDirecoes))
    matrizRetorno.append(linha)

  matrizRetorno = np.array(matrizRetorno)
  return matrizRetorno

def aplicarDescritoresDeTextura(imagem, implementacaoWavelet):
  (canalR, canalG, canalB) = separarCanaisRGB(imagem)

  coeficientesWaveletR = aplicarWavelet(canalR, implementacaoWavelet)
  coeficientesWaveletG = aplicarWavelet(canalG, implementacaoWavelet)
  coeficientesWaveletB = aplicarWavelet(canalB, implementacaoWavelet)

  coeficientesWavelet = combinarCoeficientesWavelet(coeficientesWaveletR, coeficientesWaveletG, coeficientesWaveletB)

  #exibirImagensCoeficientesWavelet(coeficientesWavelet)
  LL, (LH , HL, HH) = coeficientesWavelet
  return (aplicarGlcm(LL), aplicarGlcm(LH), aplicarGlcm(HL), aplicarGlcm(HH))

def TesteMapasDeTextura():
  imagemTeste = io.imread(root_path + "Normal/Im001.jpg")
  io.imshow(imagemTeste)
  io.show()

  mapasDeTextura = aplicarDescritoresDeTextura(imagemTeste, "haar")
  print(mapasDeTextura)

/usr/local/lib/python3.8/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


Mounted at /content/gdrive


In [17]:
from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

root_path = '/content/gdrive/My Drive/Colab Notebooks/Trabalho/'

amostrasNormais = []
for i in range(1, 256):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Normal/" + nomeArquivo)
  LL,_,_,_ = aplicarDescritoresDeTextura(imagem, "haar")
  amostrasNormais.append(LL.flatten())

amostrasGlaucomatosas = []
for i in range(256, 456):
  nomeArquivo = "Im" + str(i).rjust(3, "0") + ".jpg"
  imagem = io.imread(root_path + "Glaucoma and glaucoma suspicious/" + nomeArquivo)
  LL,_,_,_ = aplicarDescritoresDeTextura(imagem, "haar")
  amostrasGlaucomatosas.append(LL.flatten())

kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0)

quotaTreinamentoNormais = round(0.8 * len(amostrasNormais))
quotaTreinamentoGlaucomatosas = round(0.8 * len(amostrasGlaucomatosas))
amostras = amostrasNormais[:quotaTreinamentoNormais] + amostrasGlaucomatosas[:quotaTreinamentoGlaucomatosas]
saidaAmostras = [0 for i in range(quotaTreinamentoNormais)] + [1 for i in range(quotaTreinamentoGlaucomatosas)]
gpc.fit(amostras, saidaAmostras)

amostrasTeste = amostrasNormais[quotaTreinamentoNormais:] + amostrasGlaucomatosas[quotaTreinamentoGlaucomatosas:]
saidaAmostrasTeste = [0 for i in range(len(amostrasNormais) - quotaTreinamentoNormais)] + [1 for i in range(len(amostrasGlaucomatosas) - quotaTreinamentoGlaucomatosas)]

print(gpc.score(amostrasTeste, saidaAmostrasTeste))

print(len(amostras))
print(len(amostrasTeste))

0.5604395604395604
364
91
